In [1]:
using Pkg
using Convex
using SCS
using XLSX
using DataFrames
using Plots
using CSV
using Statistics
using Images
using DelimitedFiles

┌ Info: Precompiling XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1317


### 📈 Problem 1 Portfolio investment.
Our first problem will be an investment problem. We will look at stock prices from three companies and decide how to spend an amount of $1000 on these three companies. Let's first load some data.

In [2]:
T = DataFrame(XLSX.readtable("data/stock_prices.xlsx","Sheet2")...)

,MSFT,FB,AAPL
,Any,Any,Any
1,101.93,137.95,148.26
2,102.8,143.8,152.29
3,107.71,150.04,156.82
4,107.17,149.01,157.76
5,102.78,165.71,166.52
6,105.67,167.33,170.41
7,108.22,162.5,170.42
8,110.97,161.89,172.97
9,112.53,162.28,174.97


`T` is a `DataFrame` that has weekly stock prices value of three companies (Microsoft, Facebook, Apple) from the period of January 2019 - March 2019. We will first take a quick look at these prices in a quick plot.

In [1]:
plot(T[!,:MSFT],label="Microsoft")
plot!(T[!,:AAPL],label="Apple")
plot!(T[!,:FB],label="FB")

LoadError: UndefVarError: T not defined

In [2]:
# convert the prices to a Matrix to be used later in the optimization problem
prices_matrix = Matrix(T)

LoadError: UndefVarError: T not defined

To compute the weekly return, we will use the formula: `R[i,t] = (price[i,t] - price[i,t-1])/price[i,t-1]`. This is the return of stock `i` from week `t`.

In [3]:
M1 = prices_matrix[1:end-1,:]
M2 = prices_matrix[2:end,:]
R = (M2.-M1)./M1

LoadError: UndefVarError: prices_matrix not defined

Now let's assume that the vector `x = [x1 x2 x3]` will contain the total number of dollars we will invest in these companies, i.e. `x1` is how much we will invest in the first company (MSFT), `x2` is how much we will invest in FB, and `x3` is how much we will invest in `AAPL`. The return on the investment will be `dot(r,x)`, where `r = [r1 r2 r3]` is the return from each of the companies.

Here, `r` is a random variable and we will have to model it in terms of expected values. And the expected value `E(dot(r,x))` will be `E[dot(mean(R,dims=2),x)`. If we want a return of `10%` or more, then we need `dot(r,x) >= 0.1`.

Next, we will model the risk matrix. We will skip the derivation of the risk matrix here, but you can read about it here: https://www.kdnuggets.com/2019/06/optimization-python-money-risk.html. The risk matrix will be the covariance matrix of the computed return prices (`R`).

In [4]:
risk_matrix = cov(R)

LoadError: UndefVarError: cov not defined

In [5]:
# note that the risk matrix is positive definite
isposdef(risk_matrix)

LoadError: UndefVarError: isposdef not defined

In [6]:
r = mean(R,dims=1)[:]

LoadError: UndefVarError: mean not defined

Now let's solve the following problem: Someone gives you $1000 and tells you to spend them in the form of investment on these three compnaies such that you get a return of 2\% on what you spent.

The goal will be to minimize the risk, that is x'\*risk_matrix\*x.
The constraints will be 
- `sum(x) = 1`, we will compute the percentage of investment rather than the exact amount
- `dot(r,x) >= 0.02`
- `x[i] >= 0`

This problem is a convext problem, and we will use `Convex.jl` to it.

In [7]:
x = Variable(length(r))
problem = minimize(x'*risk_matrix*x,[sum(x)==1;r'*x>=0.02;x.>=0])

LoadError: UndefVarError: r not defined

Note the `Convex.NotDcp` in the answer above and the warning. `Convex.jl` requires that we pass Dcp compliant problem (Disciplined convex programming). Learn more about the DCP ruleset here: http://cvxr.com/cvx/doc/dcp.html

In [8]:
# make the problem DCP compliant
problem = minimize(Convex.quadform(x,risk_matrix),[sum(x)==1;r'*x>=0.02;x.>=0])

LoadError: UndefVarError: Convex not defined

In [9]:
solve!(problem, SCS.Optimizer)

LoadError: UndefVarError: SCS not defined

In [10]:
x

LoadError: UndefVarError: x not defined

In [11]:
sum(x.value)

LoadError: UndefVarError: x not defined

In [12]:
# return 
r'*x.value 

LoadError: UndefVarError: r not defined

In [13]:
x.value .* 1000

LoadError: UndefVarError: x not defined

The conclusion is that we should invest **67.9USD in Microsoft**, **122.3USD in Facebook**, and **809.7USD in Apple**.

---
### 🖼️ Problem 2 Image recovery.
In this problem, we are given an image where some of the pixels have been altered. The goal is to recover the unknonwn pixels by solving an optimization problem. Let's first load the figure.

In [14]:
Kref = load("data/khiam-small.jpg")

LoadError: UndefVarError: load not defined

We will convert the image to gray scale and disrupt some of the pixels

In [15]:
K = copy(Kref)
p = prod(size(K))
missingids = rand(1:p,400)
K[missingids] .= RGBX{N0f8}(0.0,0.0,0.0)
K
Gray.(K)

LoadError: UndefVarError: Kref not defined

In [16]:
Y = Float64.(Gray.(K));

LoadError: UndefVarError: Gray not defined

Given this image, the goal is now to complete the matrix. We will use a common technique for this problem developed by Candes and Tao. The goal will be to create a new matrix `X` where we minimize the nuclear norm of `X` (i.e. the sum of the singular values of `X`), and such that the entries that are already known in `Y` remain the same in `X`. We will again use `Convex.jl` to solve this problem. Let's write it down below.

In [17]:
correctids = findall(Y[:].!=0)
X = Convex.Variable(size(Y))
problem = minimize(nuclearnorm(X))
problem.constraints += X[correctids]==Y[correctids]

LoadError: UndefVarError: Y not defined

In [18]:
solve!(problem, SCS.Optimizer(eps=1e-3, alpha=1.5))

LoadError: UndefVarError: SCS not defined

In [19]:
@show norm(float.(Gray.(Kref))-X.value)
@show norm(-X.value)
colorview(Gray, X.value)

LoadError: UndefVarError: Gray not defined

In [20]:
@show norm(float.(Gray.(Kref))-X.value)
@show norm(-X.value)
colorview(Gray, X.value)

LoadError: UndefVarError: Gray not defined

---
### 🥒 Problem 3 Diet optimization problem.
This is a common problem in Numerical Optimization, and you can find multiple references about it online. Here, we will use one of the examples in the JuMP package. Refer to this page for details: https://github.com/JuliaOpt/JuMP.jl/blob/master/examples/diet.jl.

In this porblem we are given constraints on the number of (minimum, maximum) number of calories, protein, fat, and sodium to consume. We will first build a JuMP container to store this information and pass it as constraints later.

In [21]:
using JuMP
using GLPK

LoadError: ArgumentError: Package JuMP [4076af6c-e467-56ae-b986-b466b2749572] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [22]:
category_data = JuMP.Containers.DenseAxisArray(
    [1800 2200;
     91   Inf;
     0    65;
     0    1779], 
    ["calories", "protein", "fat", "sodium"], 
    ["min", "max"])

LoadError: UndefVarError: JuMP not defined

You can think of this matrix as indexed by rows via the vector `["calories", "protein", "fat", "sodium"]`, and indexed by columns via the vector `["min", "max"]`. In fact, we can now checkout the values: `category_data["calories","max"]` or `category_data["fat","min"]`.

In [23]:
@show category_data["calories","max"] 
@show category_data["fat","min"]
;

LoadError: UndefVarError: category_data not defined

Next, we will encode some information about food data we have.

In [24]:
foods = ["hamburger", "chicken", "hot dog", "fries", "macaroni", "pizza","salad", "milk", "ice cream"]

# we will use the same concept we used above to create an array indexed 
# by foods this time to record the cost of each of these items
cost = JuMP.Containers.DenseAxisArray(
    [2.49, 2.89, 1.50, 1.89, 2.09, 1.99, 2.49, 0.89, 1.59],
    foods)

LoadError: UndefVarError: JuMP not defined

Next we will create a new matrix to encode the calories,protein, fat, and sodium present in each of these foods. This will be a matrix encoded by foods by rows, and `["calories", "protein", "fat", "sodium"]` by columns.

In [25]:
food_data = JuMP.Containers.DenseAxisArray(
    [410 24 26 730;
     420 32 10 1190;
     560 20 32 1800;
     380  4 19 270;
     320 12 10 930;
     320 15 12 820;
     320 31 12 1230;
     100  8 2.5 125;
     330  8 10 180], 
    foods, 
    ["calories", "protein", "fat", "sodium"])

@show food_data["chicken", "fat"]
@show food_data["milk", "sodium"]
;

LoadError: UndefVarError: JuMP not defined

And now, we will build the model.

In [26]:
# set up the model
model = Model(GLPK.Optimizer)

categories = ["calories", "protein", "fat", "sodium"]

# add the variables
@variables(model, begin
    # Variables for nutrition info
    category_data[c, "min"] <= nutrition[c = categories] <= category_data[c, "max"]
    # Variables for which foods to buy
    buy[foods] >= 0
end)

# Objective - minimize cost
@objective(model, Min, sum(cost[f] * buy[f] for f in foods))

# Nutrition constraints
@constraint(model, [c in categories],
    sum(food_data[f, c] * buy[f] for f in foods) == nutrition[c]
)

LoadError: UndefVarError: GLPK not defined

And finally, all what's left to be done is to solve the problem

In [27]:
JuMP.optimize!(model)
term_status = JuMP.termination_status(model)
is_optimal = term_status == MOI.OPTIMAL
@show JuMP.primal_status(model) == MOI.FEASIBLE_POINT
@show JuMP.objective_value(model) ≈ 11.8288 atol = 1e-4

LoadError: UndefVarError: JuMP not defined

And to actually look at the solution, we can look at the `buy` variable.

In [28]:
hcat(buy.data,JuMP.value.(buy.data))

LoadError: UndefVarError: buy not defined

----
### 🗺️ How many passports do you need to travel the world without obtaining a visa in advance?
This problem is the same problem shown in the JuliaCon 2018 JuMP workshop, with updated code and data. The original post can be found here: https://github.com/juan-pablo-vielma/JuliaCon2018_JuMP_Workshop/blob/master/Introduction_Slides.ipynb.

We will first get the data.

In [29]:
;git clone https://github.com/ilyankou/passport-index-dataset.git

Cloning into 'passport-index-dataset'...


The file we need is `passport-index-dataset/passport-index-matrix.csv`, and we will use the `DelimitedFiles` package to read it -- this is mainly because what we are loading is a matrix and we will have to extract the matrix out of the DataFrame if we use the `CSV` package. Both are viable options, this will just be quicker.

In [30]:
passportdata = readdlm(joinpath("passport-index-dataset","passport-index-matrix.csv"),',')

LoadError: UndefVarError: readdlm not defined

These are the possible options in this matrix:

| Value | Explanation |
|---|---|
|7-360| number of visa-free days|
|VF| visa-free travel (where number of days is not applicable or known, eg freedom of movement)|
|VOA| visa on arrival|
|ETA| eTA (electronic travel authority) required|
|VR| visa required|
|-1| where passport=destination, in matrix files only|

So anything that is a number or "VF" or "VOA", can be entered without a visa in advance.

In [31]:
cntr = passportdata[2:end,1]
vf = (x ->  typeof(x)==Int64 || x == "VF" || x == "VOA" ? 1 : 0).(passportdata[2:end,2:end]);

LoadError: UndefVarError: passportdata not defined

Set up the model

In [32]:
model = Model(GLPK.Optimizer)

LoadError: UndefVarError: GLPK not defined

Add the variables, constrains, and the objective function.

In [33]:
@variable(model, pass[1:length(cntr)], Bin)
@constraint(model, [j=1:length(cntr)], sum( vf[i,j]*pass[i] for i in 1:length(cntr)) >= 1)
@objective(model, Min, sum(pass))

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[33]:1

And finally, solve the problem

In [34]:
JuMP.optimize!(model)

LoadError: UndefVarError: JuMP not defined

In [35]:
print(JuMP.objective_value(model)," passports: ",join(cntr[findall(JuMP.value.(pass) .== 1)],", "))

LoadError: UndefVarError: JuMP not defined

# Finally...
After finishing this notebook, you should be able to:
- [ ] solve optimization problems via the Convex.jl package
- [ ] solve optimization problems via the JuMP.jl package